#Instalação 

In [ ]:
# reading daaset from Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install wmd
!pip install fasttext
!pip install compress-fasttext

In [ ]:
!pip uninstall numpy
!pip install 'numpy>=1.18.0,<1.23.0'
!pip install compress-fasttext
import compress_fasttext

In [ ]:
 # Packages to store and manipulate data
import pandas as pd
import numpy as np
# Example function using numpy:
from numpy import dot
from numpy.linalg import norm
import math
import os
import json
import csv
import string
import glob
import random
import time
import math
from datetime import datetime
pd.set_option('max_colwidth', 200)


# Plotting packages
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns 
import pickle
import math
import wmd
from scipy import spatial
from scipy.spatial import distance
from scipy.spatial.distance import pdist, squareform
from collections import Counter
import plotly.express as px
%matplotlib inline
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


# Package to clean text
import re
from keras.preprocessing.text import Tokenizer
import operator
from operator import itemgetter
from operator import itemgetter
from tqdm import tqdm


# Package NPL
import nltk
#nltk.download()
nltk.download('punkt') #This is a library that helps us tokenize words and phrases
nltk.download('stopwords')
nltk.download ( 'wordnet' ) #We use the data and methods in this library to stem our data
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import brown
stop_words = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk import bigrams
from nltk.corpus import brown
from nltk.util import ngrams
from nltk import word_tokenize
from nltk import RegexpParser

from textblob import TextBlob
import spacy
nlp = spacy.load('en_core_web_sm')
from scipy.sparse import csr_matrix, issparse  # , todense
import sys
from collections import defaultdict
import multiprocessing
import fasttext


from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Input, Embedding, Dense, GRU, Dropout, Reshape, Bidirectional
from keras.callbacks import Callback, ModelCheckpoint

In [ ]:
import compress_fasttext
small_model = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
'https://github.com/avidale/compress-fasttext/releases/download/gensim-4-draft/ft_cc.en.300_freqprune_400K_100K_pq_300.bin')

#Tópicos

In [ ]:
df_topic = pd.read_csv('/content/drive/bertopic-exp1_dia_utel.csv')
df_topic

In [ ]:
list_topics = []
topic = []
for column in df_topic:
  topic = []
  for word in df_topic[str(column)]:
    topic.append(word)
  list_topics.append(topic)

print(list_topics)

#Importar Títulos e Gerar Candidatos

In [ ]:
list_topics
lst = []
for x in list_topics:
  for y in x:
    lst.append(y)
print(lst)

In [ ]:
class Title:

    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.__lower_words()
        self.__remove_punctuation()
        self.__remove_double_spacing()
        self.__remove_numbers()
        self.__selecting_column()
        self.__tokenize_and_remove_stopwords_lemmatize()
        self.__convert_to_string()
    
    def __lower_words(self):
        self.dataframe['clean_title'] = self.dataframe['title'].apply(lambda x: x.lower())
        
    def __remove_punctuation(self):
        my_punctuation = '#!"…¨$%&\'”’(“)*+_,-./:;<=>?[\\]^_`{|}~•@'
        new_tweet = []
        for tweet in self.dataframe.clean_title:
            tweet = re.sub('['+my_punctuation + ']+', ' ', tweet)
            new_tweet.append(tweet)
        self.dataframe['clean_title'] = new_tweet
        
    def __remove_double_spacing(self):
        new_tweet = []
        for tweet in self.dataframe.clean_title:
            tweet = re.sub(r'\s+', ' ', tweet) 
            new_tweet.append(tweet)
        self.dataframe['clean_title'] = new_tweet
        
    def __remove_numbers(self):
        new_tweet = []
        for tweet in self.dataframe.clean_title:
            tweet = re.sub('([0-9]+)', '', tweet)
            tweet = tweet.strip()
            new_tweet.append(tweet)
        self.dataframe['clean_title'] = new_tweet

    def __selecting_column(self):
        self.dataframe.dropna(subset=['clean_title'], axis=0, inplace = True)
        self.clean_title = self.dataframe['clean_title']
        self.text = " ".join(s for s in self.clean_title).lower()
        
    def __tokenize_and_remove_stopwords_lemmatize(self,bigrams=False):
        lemmatiser = WordNetLemmatizer()
        self.stop_words = stopwords.words('english')
        self.stop_words.extend(lst) 
        self.stop_words.extend(['list', 'redirect', 'category', 'section', 'from'])

        new_tweet = []
        for tweet in self.dataframe.clean_title:
            tweet_token = [word for word in tweet.split(' ') if len(word)> 2] # Removing short words - less than three
            tweet_token_list = [word for word in tweet_token if word not in self.stop_words]
            new_tweet.append(tweet_token_list)
        self.dataframe['clean_title_tokenize'] = new_tweet
    
    def __convert_to_string(self):
        self.new_tweet = []
        for tweet_token_list in self.dataframe.clean_title_tokenize:
            tweet = ' '.join(tweet_token_list)
            self.new_tweet.append(tweet)
        self.dataframe['clean_title'] = self.new_tweet

    def get_tokens_set(self):
      tokens = []
      for tweet in self.dataframe.clean_title:
        tokens_set = tweet.split()
        for token in tokens_set:
          tokens.append(token)
      return tokens
      
    def get_new_dataframe(self):
        return self.dataframe
        
df_label_title= pd.read_csv("/content/drive/Bertopic exp1.train.csv", index_col=False, dtype='unicode')


label_topic = Title(df_label_title)
df_label = label_topic.get_new_dataframe()
tokens = label_topic.get_tokens_set()

In [ ]:
# Count the total number of occurences of each word
word_counts = {}
for title in df_label.clean_title:
    for word in title.split():
        if word not in word_counts:
            word_counts[word] = 1
        else:
            word_counts[word] += 1
print("Size of Vocabulary:", len(word_counts.keys()))

In [ ]:
word_counts = {k: v for k, v in sorted(word_counts.items(),reverse=True, key=lambda item: item[1])}

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_label.clean_title)
word_index = tokenizer.word_index

In [ ]:
new_tokens = []
for t in tokens:
  types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
  tag = nltk.pos_tag([t])
  if tag[0][1] in types:
    new_tokens.append(t)
    
len(new_tokens)

In [ ]:
import nltk
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(new_tokens)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(new_tokens)

#bigrams
bigram_freq = bigramFinder.ngram_fd.items()
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)
#trigrams
trigram_freq = trigramFinder.ngram_fd.items()
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)

#get english stopwords
stopwords_en = set(stopwords.words('english'))

        #JJ  - adjective ‘cheap’ 
        #JJR  - adjective, comparative ‘cheaper’ 
        #JJS  - adjective, superlative ‘cheapest’
        #NN   - noun, singular ‘table’ 
        #NNS  - noun plural ‘undergraduates’ 
        #NNP  - proper noun, singular ‘Rohan' 
        #NNPS - proper noun, plural ‘Indians’ 

        #Bigrams: (Noun, Noun), (Adjective, Noun)
        #Trigrams: (Adjective/Noun, Anything, Adjective/Noun)

#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '-pron-' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in stopwords_en or word.isspace():
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

#function to filter for trigrams
def rightTypesTri(ngram):
    if '-pron-' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in stopwords_en or word.isspace():
            return False
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS') ########
    third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in first_type and tags[1][1] in second_type and tags[2][1] in third_type:
        return True
    else:
        return False

#filter bigrams
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda new_tokens: rightTypes(new_tokens))]
#filter trigrams
filtered_tri = trigramFreqTable[trigramFreqTable.trigram.map(lambda new_tokens: rightTypesTri(new_tokens))]


#filter for only those with more than 2 occurences
bigramPMITable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.pmi)), columns=['candidate','PMI']).sort_values(by='PMI', ascending=False)
trigramPMITable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.pmi)), columns=['candidate','PMI']).sort_values(by='PMI', ascending=False)

candidates = pd.concat([bigramPMITable, trigramPMITable]).sort_values(by='PMI', ascending=False)
print(len(candidates)) ##

candidates = candidates[(candidates['PMI']>4)]

list_candidate = []
string_candidate = []
for candidate in candidates.candidate:
  list_unit = []
  ngram = " ".join(candidate)
  list_unit.append(ngram)
  list_candidate.append(list_unit)
  string_candidate.append(ngram)

candidates["candidates_string"] = string_candidate
candidates["candidates_list"] = list_candidate
candidates.reset_index(drop=True, inplace=True)
candidates 

#Rótulo

In [ ]:
def calculate_cosine_distance(a, b):
    cosine_distance = float(spatial.distance.cosine(a, b))
    return cosine_distance

def calculate_cosine_similarity(a, b):
    cosine_similarity = 1 - calculate_cosine_distance(a, b)
    return cosine_similarity

def calculate_angular_distance(a, b):
    cosine_similarity = calculate_cosine_similarity(a, b)
    angular_distance = math.acos(cosine_similarity) / math.pi
    return angular_distance

def calculate_angular_similarity(a, b):
    angular_similarity = 1 - calculate_angular_distance(a, b)
    return angular_similarity

def distance(v1, v2):
    return np.sqrt(np.sum((v1 - v2) ** 2))  

def minmax_norm(df_input):
    return (df_input - df_input.min()) / ( df_input.max() - df_input.min()) 

def word_mover_similarity(a, b):
    # your distance score needs to be converted to a similarity score
    similarity = TODO_IMPLEMENT(a, b)
    return similarity

In [ ]:
name = ''
dic_cand = {} #soma das distancias de cada palavra para calcular com os tokens
# (x + y + z)/3 dist a

for n in range(0, 10):
  name = 'Topic '+ str(n)
  dic_cand[name] = {}
  dic_cand[name]['cosine'] = {}
  med_cos = {}

  for c in candidates.candidate:
    test = 0
    cal_cos = []
    for i in range(len(c)):
      primeiro = small_model[c[i]]
      test += primeiro
    media = test/len(c)

    for t in df_topic[name]:
      array_t = small_model[t]
      cos_sim = calculate_cosine_similarity(media, array_t)
      cal_cos.append(cos_sim)

    sum_cos = 0
    for i in cal_cos:
      sum_cos += i
    med_cos[sum_cos/10] = c
    dic_cand[name]['cosine'] = med_cos

In [ ]:
name = ''
dic_cand_filter = {}

for n in range(0, 10):
  name = 'Topic '+ str(n)
  dic_cand_filter[name] = {}
  a = sorted(dic_cand[name]['cosine'].items(), reverse=True) #classificando os com valores maiores
  edit_a = a[:200]
  lista_name = []

  for i in edit_a:
    lista_name.append(i[1])
  dic_cand_filter[name] = lista_name

In [ ]:
name = ''
dic_cand_best = {}

for n in range(10):
  name = 'Topic '+ str(n)
  dic_cand_best[name] = {}
  dic_cand_best[name]['cosine'] = {} 
  dic_cand_best[name]['angular'] = {}
  dic_cand_best[name]['euclidean'] = {}

  med_cos = {}
  med_ang = {}
  med_euclidean = {}

  for c in dic_cand_filter[name]:
    test = 0
    for i in range(len(c)):
      primeiro = small_model[c[i]]
      test += primeiro
    media = test/len(c)
    str_c = ' '.join(c)

    cal_cos = []
    cal_ang = []
    cal_euclidean = []

    for t in df_topic[name]:
      array_t = small_model[t]

      cos_sim = calculate_cosine_similarity(media, array_t)
      ang_sim = calculate_angular_similarity(array_t, media)
      euclidean = np.linalg.norm(array_t - media)

      cal_cos.append(cos_sim)
      cal_ang.append(ang_sim)
      cal_euclidean.append(euclidean)

    sum_cos = 0
    sum_jacc = 0
    sum_ang = 0
    sum_euclidean = 0

    for i in cal_cos:
      sum_cos += i
    for j in cal_ang:
      sum_ang += j
    for w in cal_euclidean:
      sum_euclidean += w

    med_cos[str_c] = sum_cos/10
    med_ang[str_c] = sum_ang/10
    med_euclidean[str_c] = sum_euclidean/10

    dic_cand_best[name]['cosine'] = med_cos
    dic_cand_best[name]['cosine2'] = med_cos
    dic_cand_best[name]['angular'] = med_ang
    dic_cand_best[name]['euclidean'] = med_euclidean

  df_cand_select = pd.DataFrame(data=dic_cand_best[name])
  df_cand_select['value'] = df_cand_select.sum(axis=1)/len(dic_cand_best[name].keys())
  df_cand_select = df_cand_select.reset_index()

In [ ]:
dic_cand_final = pd.DataFrame()

topic_num = []
topic_word = []
topic_label = []

for n in range(0, 10):
  name = 'Topic '+ str(n)
  topic_num.append(name)


  df_cand_select = pd.DataFrame(data=dic_cand_best[name])
  df_cand_select['cosine2'] = df_cand_select['cosine']
  df_cand_select = minmax_norm(df_cand_select)
  df_cand_select['value'] = df_cand_select.sum(axis=1)/len(dic_cand_best[name].keys())
  df_cand_select = df_cand_select.reset_index()

  best_med = 0
  cand_name = ''
  for i in range(len(df_cand_select['value'])):
      if df_cand_select['value'][i] > best_med:
        best_med = df_cand_select['value'][i]
        cand_name = df_cand_select['index'][i]

  tokens_lda = []
  for t in df_topic[name]:
    tokens_lda.append(t)

  topic_word.append(tokens_lda)

  tokens_lda_str = ", ".join(tokens_lda)
  print(str(name) +": "+str(tokens_lda_str))

  df_cand_select  = df_cand_select.sort_values(by='value', ascending=False).head(5)
  l = []
  for i in df_cand_select[:1]['index']:
    l.append(i)
    topic_label.append(i)
  l_str = ", ".join(l)
  print("candidates: " +str(l_str))
  print("\n")

dic_cand_final['Indice'] = topic_num
dic_cand_final['Tópico'] = topic_word
dic_cand_final['Rótulo'] = topic_label

In [ ]:
dic_cand_final